In [3]:
import time # 딜레이를 사용하기 위한 라이브러리
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
# 키보드를 제어하는 라이브러리
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [6]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
# 웹사이트의 모든 자원 로드를 기다리기 위해 암묵적으로 5초 대기. 그 전에 로딩이 끝나면 5초가 안 됐어도 다음 코드로 넘어감
driver.implicitly_wait(5)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - There is no [mac64] chromedriver for browser 102.0.5005 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/jason/.wdm/drivers/chromedriver/mac64/102.0.5005.61]
/var/folders/cl/n2f97vzx4f18h9g56wnqv8940000gn/T/ipykernel_4273/3424604734.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False
def remove_tags(element):
    result = ''
    try:
        result = re.sub('<.+?>', '', str(element), 0).strip()
        return result
    except NoSuchElementException:
            return ''

In [8]:
keyword=input("검색어를 입력하세요.")
baseUrl='https://www.daangn.com'
url = baseUrl+'/search/'+keyword
driver.get(url)
html = ''
result_flea = True
try:
    check_exists_by_xpath('//*[@id="flea-market-wrap"]')
    result_flea = True
except:
    result_flea = False
    
if(result_flea == True) :
    count = 0 
    while (check_exists_by_xpath('//*[@id="result"]/div[1]/div[2]') == True) :
        try:
            driver.find_element_by_xpath('//*[@id="result"]/div[1]/div[2]').click()
            time.sleep(0.5)
        except:
            html = bs(driver.page_source, 'html.parser')
            break
            
html = bs(driver.page_source, 'html.parser')

#가끔 클릭이되는 오류가 있음

검색어를 입력하세요.해피해킹


/var/folders/cl/n2f97vzx4f18h9g56wnqv8940000gn/T/ipykernel_4273/1442661154.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(xpath)
/var/folders/cl/n2f97vzx4f18h9g56wnqv8940000gn/T/ipykernel_4273/407812930.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="result"]/div[1]/div[2]').click()


In [13]:
html

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<link href="https://www.daangn.com/articles/243632612" rel="canonical"/>
<title>해피해킹 하이브리드 type-s... | 당근마켓 중고거래</title>
<meta content="해피해킹 하이브리드 type-s 판매합니다. 블랙 무각입니다. 박스풀이고 구매하고 몇번 사용안해서 사..." name="description"/>
<meta content="해피해킹, 하이브리드, type, s, 블, 랙, 무, 각인" name="keywords"/>
<link href="아이즌" rel="author"/>
<meta content="https://www.daangn.com/articles/243632612" property="og:url"/>
<meta content="해피해킹 하이브리드 type-s... | 당근마켓 중고거래" property="og:title"/>
<meta content="해피해킹 하이브리드 type-s 판매합니다.
블랙 무각입니다. 
박스풀이고 구매하고 몇번 사용안해서 사..." property="og:description"/>
<meta content="당근마켓" property="og:site_name"/>
<meta content="https://dnvefa72aowie.cloudfront.net/origin/article/202106/6F2776197C93064DFCF99A884DCB9243F12C6649D43E606E0ECBFEDC26FAC6C5.jpg?q=95&amp;s=1440x1440&amp;t=inside"

In [11]:
products = html.select('#flea-market-wrap > article')

prod_list = []

for product in products :
    prod_list.append(product)

datas = []
for prod in prod_list :
    container = dict()
    try:
        container['title'] = remove_tags(prod.find('img')['alt'])
        container['img'] = prod.find('img')['src']
        container['content'] = remove_tags(prod.find('span', {'class' : 'article-content'}))
        container['region'] = remove_tags(prod.find('p', {'class' : 'article-region-name'}))
        container['price'] = remove_tags(prod.find('p', {'class' : 'article-price'}))
        container['link'] = baseUrl+remove_tags(prod.find('a', {'class' : 'flea-market-article-link'})['href'])
    except :
        print('error', prod)
    datas.append(container)

# print(datas)

In [12]:
df = pd.DataFrame(datas)
# csv파일로 저장
df.to_csv('./'+keyword+'.csv', index=False)
# 데이터 확인
df

""
